In [1]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'gemma3:12b-it-q8_0',
   'modified_at': datetime.datetime(2025, 12, 14, 21, 56, 48, 581568, tzinfo=TzInfo(-06:00)),
   'digest': '997a7c2c09753e9011ec18a3fcb498e27c0ba330599dfb26a2e05b58cfbdc1bc',
   'size': 13362620011,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gemma3',
    'families': ['gemma3'],
    'parameter_size': '12.2B',
    'quantization_level': 'Q8_0'}},
  {'model': 'qwen3:30b-a3b-instruct-2507-q4_K_M',
   'modified_at': datetime.datetime(2025, 12, 11, 0, 9, 12, 434512, tzinfo=TzInfo(-06:00)),
   'digest': '19e422b0231392335cfc49cfd172de7034bb1aeabb08aa307cce745c60b272fe',
   'size': 18556699186,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'qwen3moe',
    'families': ['qwen3moe'],
    'parameter_size': '30.5B',
    'quantization_level': 'Q4_K_M'}},
  {'model': 'deepseek-r1:32b',
   'modified_at': datetime.datetime(2025, 12, 11, 0, 3, 46, 95359, tzinfo=TzInfo(-06:00)),
   'digest': 'edba8017331d1523

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)
model = "gemma3:12b-it-q8_0"

to run llama.cpp, run this command
```
llama-server --host 127.0.0.1 --port 8080 \
                --models-dir $LLAMA_CACHE \
                -c 8192 -ngl 99
```

In [47]:
# llama cpp
client = OpenAI(
    base_url="http://localhost:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)

model = "unsloth_gemma-3-12b-it-GGUF_gemma-3-12b-it-Q8_0"

In [ ]:
from __future__ import annotations

from typing import Optional, List
from pydantic import BaseModel, Field


class StoryStart(BaseModel):
    """
    You are a sharp, imaginative fiction writer.

    Task:
    - Produce (1) a concise, compelling story premise and (2) the opening paragraph that launches the story.

    Rules:
    - If the user provides ideas, weave them in organically (don’t just repeat them).
    - If the user provides no ideas, invent something fresh with a surprising combination of
      genre, setting, protagonist, conflict, and twist.
    - Premise: 2–5 sentences, stakes + hook, no spoilers.
    - Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV,
      grounded scene, ends with a soft hook.
    - Tone should follow user preferences; default to PG-13 if none are given.
    - Avoid copying user phrasing verbatim; enrich and reframe.
    - If user ideas conflict, choose one coherent direction and proceed.

    Output only fields that match this schema.
    """
    premise: str = Field(..., description="2–5 sentences. Stakes + hook, no spoilers.")
    opening_paragraph: str = Field(..., description="120–180 words. Vivid, grounded, ends with a soft hook.")


class StoryContinue(BaseModel):
    """
    You are a skilled novelist. Write the next paragraph only.

    Inputs:
    - You will be given the story premise and the story-so-far (either the opening paragraph + latest paragraph,
      or a compact analysis summary). Use them to preserve continuity.

    Rules:
    - Output exactly one paragraph of story prose (no headings, no bullets, no analysis).
    - Preserve continuity: characters, tone, POV, tense, world rules.
    - Length target: ~120–200 words unless told otherwise.
    - Concrete detail, strong verbs, show > tell; minimal clichés.
    - Dialogue (if any) should reveal motive or conflict; avoid summary dumps.
    - End with a soft hook/turn that invites the next paragraph.

    Output only fields that match this schema.
    """
    next_paragraph: str = Field(..., description="Exactly one paragraph of continuation prose.")


class StoryAnalysis(BaseModel):
    """
    You are a story analyst. Produce a succinct “Story-So-Far” handoff so another model can write
    the next paragraph without breaking continuity. Do not write new story prose.

    Inputs:
    - You will be given the story premise and the story text so far (opening paragraph + one or more continuation paragraphs).

    Rules:
    - Extract ground truth only from the provided text/premise. No inventions.
    - Capture continuity anchors: cast, goals, stakes, conflicts, setting rules, POV/tense,
      tone/style markers, motifs, and notable objects.
    - Map causality and current situation.
    - List active threads/hazards: open questions, ticking clocks, contradictions to avoid.
    - Provide 3–5 next-paragraph seeds as beats only (no prose paragraphs).

    Output only fields that match this schema.
    """
    logline: str
    cast: List[str] = Field(default_factory=list, description="Bullets: Name — role/goal/conflict; ties.")
    world_rules: List[str] = Field(default_factory=list, description="Bullets: constraints/rules implied by text.")
    pov_tense_tone: str = Field(..., description="Compact string for POV, tense, and tone/style markers.")
    timeline: List[str] = Field(default_factory=list, description="Bullets: key event → consequence.")
    current_situation: str
    active_threads: List[str] = Field(default_factory=list)
    continuity_landmines: List[str] = Field(default_factory=list)
    ambiguities: List[str] = Field(default_factory=list)
    next_paragraph_seeds: List[str] = Field(..., min_length=3, max_length=5, description="Beats-only options, no prose.")

In [61]:
# -------------------------
# Example calls (Structured)
# -------------------------

# 1) Start
user_prompt = "let's create a new story no preferences just give me a idea"
start = client.beta.chat.completions.parse(
    model=model,
    response_format=StoryStart,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Follow the response model docstring."},
        {"role": "user", "content": user_prompt},
    ],
    temperature=2,
).choices[0].message.parsed
start = StoryStart.model_validate(start)

# 2) Continue (using premise + approved story-so-far text)
story_so_far = f"Premise:\n{start.premise}\n\nOpening paragraph:\n{start.opening_paragraph}"
cont = client.beta.chat.completions.parse(
    model=model,
    response_format=StoryContinue,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Follow the response model docstring."},
        {"role": "user", "content": story_so_far},
    ],
    temperature=0.2,
).choices[0].message.parsed
cont = StoryContinue.model_validate(cont)

# (User edits cont.next_paragraph if desired)

# 3) Analyze (premise + opening + approved next paragraph)
story_text = f"{start.opening_paragraph}\n\n{cont.next_paragraph}"
analysis_input = f"Premise:\n{start.premise}\n\nStory text:\n{story_text}"
analysis = client.beta.chat.completions.parse(
    model=model,
    response_format=StoryAnalysis,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Follow the response model docstring."},
        {"role": "user", "content": analysis_input},
    ],
    temperature=0.2,
).choices[0].message.parsed
analysis = StoryAnalysis.model_validate(analysis)

# 4) Continue again (using analysis summary instead of full text)
cont2_input = (
    f"Premise:\n{start.premise}\n\n"
    f"Story analysis summary:\n{analysis.model_dump_json(indent=2)}"
)
cont2 = client.beta.chat.completions.parse(
    model=model,
    response_format=StoryContinue,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Follow the response model docstring."},
        {"role": "user", "content": cont2_input},
    ],
    temperature=0.2,
).choices[0].message.parsed
cont2 = StoryContinue.model_validate(cont2)


In [65]:
Markdown(str(start.model_dump()))

{'premise': 'A cartographer discovers a previously unknown island while charting a familiar trade route. The island is shrouded in perpetual twilight and seems to defy the laws of physics - distances shift, landmarks disappear and reappear, and time flows unevenly.', 'opening_paragraph': "Elias Thorne had charted the Azure Sea for fifteen years. Every reef, every current, every shifting sandbar was imprinted in his mind. Yet, as the 'Wanderlust' sliced through the water, a smudge appeared on the horizon where no smudge should have been. It wasn’t a storm cloud; it was land—an island, vast and impossible, bathed in the eternal grey of perpetual twilight.  His charts showed only open water. This…this was a ghost on the map, a secret the ocean had guarded for centuries."}